In [36]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [37]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

dataNBA = pd.read_pickle("~/code/dataset-worldnews-2022/data/dataNBA.pkl") #load pandas dataframe dataNBA with embeddings

dataNBA['cluster'] = [[-1] for _ in range(len(dataNBA))] #koren drevesa

In [2]:
#naredi k-means, ki funkcijo ki vpiše notri v pdDataframe v kateri iteracija tree-ja je clusteriral clusterje
#column clusters more vsebovat liste!!!
def KmeansCluster(Dataframe):
    embeddings = Dataframe["embeddings"].tolist() #from dataframeget embeddings
    clusters = KMeans(n_clusters=2, random_state=42).fit(embeddings)
    clusters = clusters.labels_.tolist()
    
    addToClusterColumn(Dataframe, clusters) #this will add to column cluster which is a list the cluster number in which the DOC was clustered
    
    dataOne = Dataframe[Dataframe.cluster.str[-1] == 0] #(pridobi zadnje število v listu ter loči v svoj dataframe glede na to ali je bil clustered v 0 ali 1)
    dataTwo = Dataframe[Dataframe.cluster.str[-1] == 1] #get the last number in the list (telling us the number of the cluster) to create two separate datasets
    
    return dataOne, dataTwo, silhouette(clusters, embeddings)

In [3]:
def silhouette(clusters, embeddings):
    score = silhouette_score(embeddings, clusters, metric = "cosine")
    if score <= 0.8:
        return True
    return False

In [4]:
def condition(Dataframe):
    NumOfDocs = Dataframe.shape[0] #pridobi velikost dataframe
    if NumOfDocs >= 10:
        return True
    return False

In [5]:
def addToClusterColumn(Dataframe, clusters):
    y = 0
    for val in Dataframe.cluster: #val is a list
        val.append(clusters[y])
        y+=1

In [39]:
def cluster(Dataframe):
    dataOne, dataTwo, condi = KmeansCluster(Dataframe)
    if condition(dataOne) and condi: #popravi če je velkost clusterja enaka 0 --> prekini
        cluster(dataOne)
    if condition(dataTwo) and condi:
        cluster(dataTwo)

In [40]:
#create clusters with k-means (TREE)
cluster(dataNBA)

In [8]:
#dataNBA

In [41]:
def MaxListLength(Dataframe):
    length = 0
    y = 0
    for val in Dataframe.cluster:
        if len(val) > length:
            length = len(val)
        y+=1
    return length

In [42]:
#this is used to create a unique decimal number by adding zeroos to binary numbers that are > of the MAX binary number
def ZerosToList(mylist, maxLength):
    x = len(mylist)
    N = maxLength - x
    my_array = np.asarray(mylist)
    my_array = np.pad(my_array, (0, N), 'constant')
    mylist = list(my_array)
    return mylist

In [43]:
def clusterNumb(Dataframe):   #first pop, then add zeros at the end of lists, binary to decimal #jih ne dejankso doda ampak doda da ddobi bynarno nato samo rezultat doda.
    maxListLen = MaxListLength(Dataframe)
    y = 0
    for val in Dataframe.cluster:
        val.pop(0) #delete frist number (-1) from list
        #Binar = ZerosToList(val, maxListLen)
        #res = int("".join(str(x) for x in Binar), 2)
        #val.append(res)
        y+=1

In [44]:
clusterNumb(dataNBA) #appenda enolično desetiško število v column cluster

In [45]:
# From a list of binary numbers it creates a string rapresenting a binary number wich is the CLUSTER ID 
dataNBA['cluster'] = dataNBA['cluster'].apply(lambda x: ''.join(map(str, x)))

# TIME window (Binary cluster ID-s only)

In [74]:
from datetime import timedelta

def TimeWindow(dataframe, IndexDateLIst, NumOfDays):
    max_window_start = None
    max_window_end = None
    max_window_count = 0

    # Sort the list of tuples by date
    sorted_list = sorted(IndexDateLIst, key=lambda x: x[1])
    #print(sorted_list)

    # Loop through the sorted list of tuples and count the number of indexes within each 5-day window
    for i in range(len(sorted_list)):
        window_start = sorted_list[i][1]
        window_end = window_start + timedelta(days=NumOfDays)
        window_count = 1
        for j in range(i+1, len(sorted_list)):
            if sorted_list[j][1] >= window_end:
                break
            window_count += 1

        if window_count > max_window_count:
            max_window_count = window_count
            max_window_start = window_start
            max_window_end = window_end


    # Print the results
    #print("The window with the most indexes is from",
    #      max_window_start.strftime('%Y-%m-%d'),
    #      "to",
    #      max_window_end.strftime('%Y-%m-%d'))


    #list of index-es of documents published outside the time window
    sorted_list = [x[0] for x in sorted_list if x[1] < max_window_start or x[1] > max_window_end]
    #print(sorted_list)

    #Delete elements out of dataframe
    dataframe = dataframe.drop(index=sorted_list)

    return dataframe

In [73]:
def Time(dataframe, NumOfDays):
    clusters = dataframe.cluster.unique()
    for ClusterID in clusters:
        IndexDateLIst = list(zip(dataframe[dataframe['cluster'] == ClusterID].index, dataframe[dataframe.cluster == ClusterID].date))
        dataframe = TimeWindow(dataframe, IndexDateLIst, NumOfDays) # assign the modified dataframe to a variable
    return dataframe # return the modified dataframe

In [49]:
#If you want to test something do it on the test data so the original wont be destroid
test = dataNBA.copy(deep=True)

In [75]:
test = Time(test, 5)

In [33]:
dataNBA

,uri,lang,isDuplicate,date,time,dateTime,dateTimePub,dataType,sim,url,...,authors,image,eventUri,sentiment,wgt,relevance,concepts,userHasPermissions,embeddings,cluster
4,6277667426,zho,False,2020-11-01,00:09:00,2020-11-01 00:09:00,2020-11-01 00:09:00,news,0.772549,https://sports.163.com/20/1101/07/FQB5A1SQ0005...,...,[],None,zho-885156,NaN,341885340,76,"[basketball, nba]",NaN,"[0.010078161023557186, 0.02238043211400509, -0...",-1111100011010
10,6277672319,spa,False,2020-11-01,00:18:00,2020-11-01 00:18:00,2020-10-31 23:55:00,news,0.000000,https://www.catamarcactual.com.ar/deportes/202...,...,[],https://www.catamarcactual.com.ar/u/fotografia...,None,NaN,341885880,26,"[basketball, nba]",NaN,"[0.011584332212805748, 0.010005190037190914, -...",-11100111101011
16,6277674117,spa,False,2020-11-01,00:22:00,2020-11-01 00:22:00,2020-11-01 00:15:00,news,0.000000,https://www.excelsior.com.mx/adrenalina/las-ma...,...,[],https://cdn2.excelsior.com.mx/media/styles/lar...,None,NaN,341886120,26,"[basketball, nba]",NaN,"[0.01666124351322651, 0.010164867155253887, -0...",-111110001010100111
19,6277674751,eng,False,2020-11-01,00:23:00,2020-11-01 00:23:00,2020-11-01 00:23:00,news,0.000000,https://www.msn.com/en-xl/sport/sports_news/wh...,...,[],https://img-s-msn-com.akamaized.net/tenant/amp...,None,0.215686,341886180,88,"[basketball, nba]",NaN,"[-0.011159778572618961, 0.02371053397655487, -...",-110101000110111010
22,6277677118,eng,False,2020-11-01,00:27:00,2020-11-01 00:27:00,2020-11-01 00:21:00,news,0.000000,https://heavy.com/sports/kobe-bryant-shaquile-...,...,[],https://heavy.com/wp-content/uploads/2020/01/g...,None,0.309804,341886420,76,"[basketball, nba]",NaN,"[0.010649552568793297, 0.05847709998488426, -0...",-1010000011011101000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70471,6322829864,eng,False,2020-11-30,23:49:00,2020-11-30 23:49:00,2020-11-30 23:48:00,news,0.000000,https://forums.rotoworld.com/profile/230133-to...,...,[],None,None,0.105882,344476140,39,"[basketball, nba]",NaN,"[0.03465336933732033, 0.01572950929403305, 0.0...",-100011110000111
70472,6322832198,eng,False,2020-11-30,23:51:00,2020-11-30 23:51:00,2020-11-30 23:39:00,news,0.866667,https://www.espn.com/nba/story/_/id/30423955/n...,...,[],https://a2.espncdn.com/combiner/i?img=%2Fphoto...,eng-6350546,0.254902,344476260,100,"[basketball, nba]",NaN,"[0.06188869848847389, 0.02430535852909088, -0....",-101010001001
70477,6322835719,eng,False,2020-11-30,23:55:00,2020-11-30 23:55:00,2020-11-30 23:55:00,news,0.721569,https://www.nola.com/sports/pelicans/article_5...,...,"[{'uri': 'christian_clark@nola.com', 'name': '...",https://bloximages.newyork1.vip.townnews.com/n...,eng-6355181,-0.011765,344476500,63,"[basketball, nba]",NaN,"[0.04360201954841614, 0.05989431217312813, -0....",-10100010010000
70481,6322837904,eng,False,2020-11-30,23:58:00,2020-11-30 23:58:00,2020-11-30 23:58:00,news,0.000000,http://laconfidentialmag.com/paul-pierce-house...,...,[],http://laconfidentialmag.com/get/files/image/g...,None,0.168627,344476680,51,"[basketball, nba]",NaN,"[0.03434178605675697, 0.04251549020409584, -0....",-10111010011


# Clusters infromation

In [16]:
# Number of generated clusters
dataNBA.cluster.nunique()

2372

In [31]:
def getClusterDensity(df):
    clusters = dataNBA.cluster.unique()
    for x in range(11):
        NumOfClusters = []
        for cluster in clusters:
            if len(df[df.cluster == cluster]) == x:
                NumOfClusters.append(cluster)
        print("Število novic v clusterju:",x,". Število clusterjev z toliko novicami",len(NumOfClusters))

In [72]:
getClusterDensity(test)

Število novic v clusterju: 0 . Število clusterjev z toliko novicami 0
Število novic v clusterju: 1 . Število clusterjev z toliko novicami 393
Število novic v clusterju: 2 . Število clusterjev z toliko novicami 475
Število novic v clusterju: 3 . Število clusterjev z toliko novicami 458
Število novic v clusterju: 4 . Število clusterjev z toliko novicami 367
Število novic v clusterju: 5 . Število clusterjev z toliko novicami 245
Število novic v clusterju: 6 . Število clusterjev z toliko novicami 172
Število novic v clusterju: 7 . Število clusterjev z toliko novicami 118
Število novic v clusterju: 8 . Število clusterjev z toliko novicami 87
Število novic v clusterju: 9 . Število clusterjev z toliko novicami 56
Število novic v clusterju: 10 . Število clusterjev z toliko novicami 0


# Tips how to search the clusters (with BINARY cluster ID)

In [60]:
#this will return cluster ID-s of all clusters with x number of news inside
def getClustersIds(x, df): #where x is the number of news and df is the dataset
    clusters = dataNBA.cluster.unique()
    cluster = df["cluster"]
    NumOfClusters = []
    for cluster in clusters:
        if len(df[df.cluster == cluster]) == x:
            NumOfClusters.append(cluster)
    print(NumOfClusters)

In [61]:
getClustersIds(5, test)

['10101000110111010', '100010000', '1101000101101', '010000011101', '10101000100011010', '111111110011010010', '1010011110001101', '0100000111110', '101011110001101', '0110000100100', '010110100111111', '1000001101', '101001111101000', '0010111000101110', '1000011010', '111100011110', '0001011011011', '0100010100011110', '10101000111101001', '100011010110', '0110110001', '0001111110110', '111010101111100', '111100110110', '100001110110', '010000111', '1110111111', '100001110111', '0110000100101', '010000100001', '101011111010110001', '01100111000', '111101111111', '010001101001011', '11011000110', '1010111011100', '10100001110010', '1111001011010', '0100011001000', '100001100100', '01110000101011', '0000110101101', '01010101011', '01100110011', '011001101001', '0011011010', '011111000111011', '101011111100', '000100000010', '10000011001', '11111111001110', '1100101001010', '01101000101', '010000011011100', '0100111001001', '1010111111101001', '0001111000001', '1101011010', '1010101111'

In [63]:
#All body text of selected cluster will be printed
def getAllDocCluster(BinaryString, df):
    Body = df[df.cluster == BinaryString]["body"]
    for a in Body:
        print(a)
        print("-----------------------------------------------------------------------------------------------------------------------------------")

In [ ]:
getAllDocCluster("10101000110111010", test)

In [55]:
def getDocBody(index, df):
    print(df.body[index])

In [ ]:
getDocBody(22, test)

In [68]:
# str.startswith() poišče v stringu vse, ki se začnejo v naslednjem vrstnem redu
filtered_df = test[test['cluster'].str.startswith('10101000110111011')]
filtered_df

,uri,lang,isDuplicate,date,time,dateTime,dateTimePub,dataType,sim,url,...,authors,image,eventUri,sentiment,wgt,relevance,concepts,userHasPermissions,embeddings,cluster
21967,6292141647,eng,False,2020-11-11,00:29:00,2020-11-11 00:29:00,2020-11-11 00:19:00,news,0.000000,https://nypost.com/2020/11/10/st-johns-lands-c...,...,[],https://nypost.com/wp-content/uploads/sites/2/...,None,0.160784,342750540,38,"[basketball, nba]",NaN,"[0.020228208974003792, -0.014327682554721832, ...",10101000110111011
28355,6297447858,eng,False,2020-11-14,05:20:00,2020-11-14 05:20:00,2020-11-14 05:18:00,news,0.000000,https://www.en24.news/2020/10/sue-bird-seattle...,...,[],https://www.en24.news/24/wp-content/uploads/20...,None,0.262745,343027200,26,"[basketball, nba]",NaN,"[0.07274994999170303, 0.02013142593204975, 0.0...",10101000110111011
39533,6305724760,por,False,2020-11-19,11:53:00,2020-11-19 11:53:00,2020-11-19 11:39:00,news,0.000000,https://observador.pt/2020/11/19/anthony-edwar...,...,"[{'uri': 'mariana_fernandes@observador.pt', 'n...",https://wm.observador.pt/wm/obs/l/https%3A%2F%...,None,NaN,343482780,100,"[basketball, nba]",NaN,"[0.10900672525167465, 0.021650614216923714, -0...",10101000110111011
44342,6308352122,spa,False,2020-11-21,00:09:00,2020-11-21 00:09:00,2020-11-20 23:00:00,news,0.521569,https://www.entornointeligente.com/esecutivo-j...,...,[],https://www.entornointeligente.com/wp-content/...,spa-2150210,NaN,343613340,63,"[basketball, nba]",NaN,"[0.052855394780635834, -0.010787829756736755, ...",10101000110111011
69329,6322250102,eng,False,2020-11-30,15:11:00,2020-11-30 15:11:00,2020-11-30 15:01:00,news,0.000000,http://www.tribtown.com/2020/11/30/ap-sportlig...,...,"[{'uri': 'associated_press@tribtown.com', 'nam...",None,None,0.435294,344445060,63,"[basketball, nba]",NaN,"[0.0039023428689688444, 0.04620026797056198, -...",10101000110111011
